In [ ]:
include("_.jl")

In [ ]:
hi = SE["hierarchy"]

ex = SE["example_md"]

he_ = [li for li in readlines(ex) if li != ""]

te = true

In [ ]:
println("Enforcing name format")

for (ro, di_, fi_) in walk_up(hi)

    for na in [di_; fi_]

        cl = Kwat.path.clean(na)

        if na != cl

            move(joinpath(ro, na), joinpath(ro, cl))

        end
        
        if occursin(r"^[0-9]+\.[0-9]+[_0-9a-z]+(\.md$|$)", na)

            println("Bad name? ", na)
            
        end
        
    end

end

In [ ]:
println("Enforcing name numbering")

for (ro, di_, fi_) in walk_up(hi)

    go_ = Vector{String}()

    ba_ = Vector{String}()

    for na in [di_; fi_]

        if is_good_structure(na)

            push!(go_, na)

        else

            push!(ba_, na)

        end

    end

    sort!(go_; by = split_good_structure)

    n_go = length(go_)

    for (id, na) in enumerate([go_; ba_])

        pa1 = joinpath(ro, na)

        if id <= n_go

            pa2 = joinpath(ro, replace(na, r"^[0-9]+" => id))

        else

            pa2 = joinpath(ro, join([id, na], "."))

        end

        if pa1 != pa2

            move(pa1, pa2; te = te)

        end

    end

end

In [ ]:
println("Enforcing 1._.md")

for (ro, di_, fi_) in walk_up(hi)

    for fi in fi_

        id, ti = split(fi, ".")

        pa1 = joinpath(ro, fi)

        pa2 = joinpath(ro, string("1._.md"))

        if ti == split(splitdir(ro)[2], ".")[2] || ti == "_" && id != "1"

            move(pa1, pa2; te = te)

        end

    end

end

In [ ]:
println("Checking .md content")

for (ro, di_, fi_) in walk_up(hi)

    for fi in fi_

        pa = joinpath(ro, fi)

        li_ = readlines(pa)

        if length(li_) == 0

            println("Is empty (populating): ", pa)

            cp(em, pa; force = true)

        elseif !all(he in li_ for he in he_)

            #error("Is missing at least 1 required header: ", pa)

        elseif !all(0 .< diff([findfirst(li_ .== he) for he in he_]))

            error("Has incorrectly ordered headers: ", pa)

        end

    end

end

In [ ]:
println("IDing .md")

ti_di = Dict{String,Dict{String,Union{String,Vector{Int64}}}}()

for (ro, di_, fi_) in walk_up(hi)

    for fi in fi_

        id, ti = split(fi, ".")

        if ti == "_"

            if id != "1"

                error("ID is not 1: ", fi)

            end

            ti = split(splitdir(ro)[2], ".")[2]

        end

        pa = joinpath(ro, fi)

        if haskey(ti_di, ti)

            error("Duplicated: ", ti_di[ti], " and ", pa)

        else

            ti_di[ti] = Dict("path" => pa, "id" => make_id(pa, "hierarchy"))

        end

    end

end

In [ ]:
na_ = sort(collect(keys(ti_di)))

In [ ]:
println("Checking name")

for wo in SE["words_to_check"]
    
    println("-" ^ 80)
    
    for na in na_
        
        if occursin(Regex("(?<![a-z])$wo(?![a-z])"), na)

            println(na)

        end
        
    end
    
    println()

end

In [ ]:
be_af = Dict(
    "decreased_heart_sound" => "diminished_heart_sound",
    "decreased_breath_sound" => "diminished_breath_sound",
)

println("Renaming")

for (be, af) in be_af

    for na in na_

        if occursin(be, na)

            pa = ti_di[na]["path"]

            di, fi = splitdrive(pa)

            pa2 = joinpath(di, replace(fi, be => af))

            move(pa, pa2; te = te)

        end

    end

end